In [ ]:
%load_ext autoreload
%autoreload 2

from common import *

alm = Resources["scenario", "10.alm"].content
alm_test = Resources.from_file("data", "atest.alm")

In [ ]:
# load tilemaps
import numpy as np
from src.formats import sprites

tilemap_path = ("graphics", "terrain")
tilemap_list = Resources[tilemap_path]
tilemaps = [x.record.content for x in tilemap_list]
tilemaps = [x.convert("RGB") for x in tilemaps]
array_tilemaps = [np.array(x) for x in tilemaps]

In [ ]:
TILE_COUNT = 24
w, h = alm.width, alm.height
display = np.zeros((TILE_COUNT * 32, TILE_COUNT * 32, 3), dtype="uint8")
x0 = 8
y0 = 8

for y in range(TILE_COUNT):
    for x in range(TILE_COUNT):                         
        tile_id = alm["tiles"].body.tiles[(y+y0) * w + x + x0].tile_id                                 
        col_id = tile_id >> 4
        row_id = tile_id & 0xF
        tile_image = array_tilemaps[col_id][row_id * 32:(row_id + 1) * 32]
        display[y*32:(y+1)*32, x*32:(x+1)*32] = tile_image
sprites.PILImage.fromarray(display)

In [ ]:
TILE_SIZE =  32
EXTRUDE_BORDER = 1

border = EXTRUDE_BORDER
b, ts = border, TILE_SIZE
tsb, w = ts + b, ts + b * 2

terrain_stripe_list = Resources["graphics", "terrain"]
stripes = [x.record.content for x in terrain_stripe_list]
max_h = max([x.height for x in stripes])
display = np.zeros((max_h//ts * w, 53 * w, 3), dtype='uint8')

def extrude_stripe(stripe, border=1):
    b, ts = border, TILE_SIZE
    tsb, w = ts + b, ts + b * 2

    arr = np.array(stripe.convert("RGB"))
    h = arr.shape[0]
    stripe_buff = np.zeros((w * h // ts, w, 3), dtype='uint8')
    buff = np.zeros((w, w, 3), dtype='uint8')
    for j in range(h // TILE_SIZE):
        buff[b:tsb, b:tsb] = arr[j * ts:(j + 1) * ts]
        buff[0:b, b:tsb] = arr[j * ts:j * ts + 1]
        buff[tsb:w, b:tsb] = arr[(j + 1) * ts - 1:(j + 1) * ts]
        buff[b:tsb, 0:b] = arr[j * ts:(j + 1) * ts, 0:1]
        buff[b:tsb, tsb:w] = arr[j * ts:(j + 1) * ts, ts - 1:ts]

        stripe_buff[j * w:(j + 1) * w] = buff
    return stripe_buff

# for i, stripe in enumerate(stripes[:16]):    
for i, stripe in enumerate(stripes):    
    stripe = extrude_stripe(stripe, EXTRUDE_BORDER)
    h = stripe.shape[0]    
    display[0:h, i * w: (i+1) * w] = stripe
    
img = sprites.PILImage.fromarray(display)
img
#img.save("terrain.png")

In [ ]:
img.width * img.height * 3 / 1024 / 1024

In [ ]:
img = img.quantize()

In [ ]:
img.palette.colors

In [ ]:
stripe_img = sprites.PILImage.fromarray(stripe)
len(stripe_img.convert("RGB").tobytes())